# Loading Dependencies
---

In [ ]:
import sys

try:
    from google.colab import drive
    drive.mount('/content/gdrive')

    !pip install yfinance
    !pip install pdfplumber
    !rm -rf github
    !mkdir github
    !cd github && git clone --branch InternationalWallet https://github.com/dr315/Finances.git
    sys.path.append('/content/github/Finances')
    
    root = '/content/gdrive/MyDrive/'
except:
    root = 'd:/'

import pandas as pd
pd.options.display.float_format = '${:,.2f}'.format
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from FinanceTools import *
from OrdersReader import *

ReadTDStatement(root + '/Investing/Notas_TD/transactions.csv', root + '/Investing/TD.csv')
ReadOrders(root + 'Investing/Notas_Clear', 'operations.csv', 'Clear')
try:
    os.mkdir('debug')
except:
    pass

# Loading and formating table
----

In [ ]:
# file_ = root + '/Investing/operations.csv'
# df = pd.read_csv(file_)
# df.sort_values(by=['Date', 'Type','Fee'], ascending=[True, False, True], inplace=True)
# df.to_csv(file_, index=False)

In [ ]:
# df = pd.read_csv(root + '/Investing/TD.csv')
df = pd.read_csv(root + '/Investing/operations.csv')

df = df.iloc[:, :7]
df.columns=['SYMBOL','DATE', 'PRICE', 'QUANTITY', 'OPERATION', 'TYPE', 'FEE']

#drop empty lines
df=df[df['DATE'].astype(bool)].dropna()

brTickers = np.sort(df[df['TYPE'].isin(['Ação'])]['SYMBOL'].unique()).tolist()
fiiTickers = np.sort(df[df['TYPE'] == 'FII']['SYMBOL'].unique()).tolist()
usTickers = np.sort(df[df['TYPE'].isin(['STOCK', 'REIT'])]['SYMBOL'].unique()).tolist()

if(df['PRICE'].apply(type).eq(str).any()): 
    df['PRICE'] = df['PRICE'].str.replace(',', '')
    df['PRICE'] = pd.to_numeric(df['PRICE'], errors='coerce')
    df['QUANTITY'] = pd.to_numeric(df['QUANTITY'], errors='coerce')
    df['FEE'] = pd.to_numeric(df['FEE'], errors='coerce')

if(df['DATE'].apply(type).eq(str).any()): 
    df['DATE'] = df.DATE.str.replace('-','/')
    df['DATE'] = pd.to_datetime(df['DATE'], format='%Y/%m/%d')
df['Year'] = pd.DatetimeIndex(df['DATE']).year
df['Month'] = pd.DatetimeIndex(df['DATE']).month_name()

#Sort the table by date and Type and reset index numeration
df.sort_values(by=['DATE', 'OPERATION'], ascending=[True, True], inplace=True)
df.reset_index(drop=True, inplace=True)

#turn all sell amount negative
df.loc[df.OPERATION == 'S', ['QUANTITY']] *= -1

#Get the oldest order date
startDate = df.iloc[0]['DATE']

df['AMOUNT'] = df['PRICE'] * df['QUANTITY'] 


In [ ]:
# import concurrent.futures
import time
import threading
start_time = time.time()

prcReader = PriceReader(brTickers + fiiTickers, usTickers, startDate)
splReader = SplitsReader(brTickers, usTickers, startDate)
divReader = DividendReader(brTickers, fiiTickers, usTickers, startDate)

if len(usTickers) > 0:
    divReader = YfinanceReader(None, None, usTickers, startDate)

def threadExecutor(obj):
    obj.load()

threadList = []
threadList.append(threading.Thread(target=threadExecutor, args=(prcReader,)))
threadList.append(threading.Thread(target=threadExecutor, args=(splReader,)))
threadList.append(threading.Thread(target=threadExecutor, args=(divReader,)))

for td in threadList:
    td.start()

for td in threadList:
    td.join()

print("Executed in %s seconds" % (time.time() - start_time))
splReader.df.to_csv('debug/log_spl.tsv', sep='\t')
divReader.df.to_csv('debug/log_div.tsv', sep='\t')
prcReader.df.to_csv('debug/log_pcr.tsv', sep='\t')

In [ ]:
dfBak = df

In [ ]:
df=dfBak
df['acum_qty'] = 0
df['PM'] = 0
df['CASH'] = 0
df['PAYDATE'] = df['DATE']
# df[df['SYMBOL'] == 'ITUB3']
# Clear operation costs befere 2019
# df.FEE.update(df.apply(clear2018Cost, axis=1))

## Accumulator

In [ ]:

for paper in (brTickers + fiiTickers + usTickers):
  paperTable = df[df.SYMBOL == paper]
  fromDate = paperTable.iloc[0]['DATE']  
  toDate = dt.datetime.today().strftime('%Y-%m-%d')

  divTable = divReader.getPeriod(paper,fromDate, toDate).reset_index()  
  #TODO: Date shall be used to calculate if 
  # divTable['DATE'] = divTable['PAYDATE']
  # divTable = divTable[['DATE', 'SYMBOL', 'PRICE']]

  divTable['QUANTITY'] = 0
  divTable['OPERATION'] = 'D'
  divTable['TYPE'] = paperTable.iloc[0]['TYPE']
  divTable['FEE'] = 0
  divTable['Year'] = pd.DatetimeIndex(divTable['DATE']).year
  divTable['Month'] = pd.DatetimeIndex(divTable['DATE']).month_name()
  divTable['AMOUNT'] = 0
  divTable['acum_qty'] = 0
  divTable['CASH'] = 0
  df = df.append(divTable)
  
  splitTable = splReader.getPeriod(paper,fromDate, toDate).reset_index()
  splitTable['PRICE'] = 0
  splitTable['OPERATION'] = 'SPLIT'
  splitTable['TYPE'] = paperTable.iloc[0]['TYPE']
  splitTable['FEE'] = 0
  splitTable['Year'] = pd.DatetimeIndex(splitTable['DATE']).year
  splitTable['Month'] = pd.DatetimeIndex(splitTable['DATE']).month_name()
  splitTable['AMOUNT'] = 0
  splitTable['acum_qty'] = 0
  splitTable['CASH'] = 0 
  splitTable['PAYDATE'] = splitTable['DATE'] 
  df = df.append(splitTable)

df.sort_values(['DATE', 'OPERATION'], inplace=True)

#Create obj
ta = TableAccumulator()
#Calc the average price and rename the columns names
df=df.groupby(['SYMBOL']).apply(ta.ByGroup).reset_index(drop=True).dropna()
df.sort_values(['PAYDATE', 'OPERATION'], ascending=[True, False],inplace=True)
df=df.apply(ta.Cash, axis=1)
# df=df.sort_values(['DATE', 'OPERATION']).reset_index(drop=True)

In [ ]:
# df[df.SYMBOL.str.contains('CASH')]
df.to_csv('debug/df_log.tsv', sep='\t')

# Realized Profit


## Processing

In [ ]:
def CalcProfit(dtf):
    profit = Profit()
    dtf.sort_values(by=['DATE', 'OPERATION'], ascending=[True, True], inplace=True)
    dtf.reset_index(drop=True)
    return dtf.groupby(['SYMBOL', 'DATE']).apply(profit.Trade).reset_index(drop=True)


In [ ]:
df = CalcProfit(df)
rl = df[df.OPERATION == 'S'][['DATE', 'SYMBOL', 'TYPE', 'Profit',	'DayTrade']]
rl['DATE'] = rl['DATE'].apply(lambda x: x.strftime('%Y-%m-%d'))
rl1 = rl
rl1.loc['Total', 'Profit'] = rl['Profit'].sum()
rl1 = rl1.fillna(' ')
rl1.style.applymap(color_negative_red, subset=['Profit']).format( {'Profit': '$ {:,.2f}', 'DayTrade': '{}'})


In [ ]:
rl = rl.groupby('SYMBOL').Profit.sum().reset_index()
rl.loc['Total', 'Profit'] = rl['Profit'].sum()
rl = rl.fillna(' ')
rl.style.applymap(color_negative_red, subset=['Profit']).format( {'Profit': '$ {:,.2f}'})


# Portifolio


In [ ]:
Portifolio(prcReader,df).show()

# Performance Blueprint

In [ ]:

p = PerformanceBlueprint(prcReader, df, dt.datetime.today().strftime('%m-%d-%Y'))
PerformanceViewer(p.calc()).show()


# Taxation


In [ ]:
profitLossDF = df.loc[df['Profit'] != 0]
tx = Taxation(profitLossDF)

## FII

### Swing Trade

In [ ]:
tx.Process('FII')
try:
    display(tx.swingTradeTable)
except:
  print('Not avalilable')

### Day Trade

In [ ]:
try:
  display(tx.dayTradeTable)
except:
  print('Not avalilable')

## Stocks


### Swing Trade

In [ ]:
tx.Process('Ação')
try:
    display(tx.swingTradeTable)
except:
  print('Not avalilable')    

### Day Trade

In [ ]:
try:
  display(tx.dayTradeTable)
except:
  print('Not avalilable')

# Plots

## Historical profitability

In [ ]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
periodPicker = widgets.Dropdown(options=['1 year', '2 years', 'All'], value='1 year')
periodPicker


In [ ]:
startPlot = startDate
frequency = 'SM'

if(periodPicker.value != 'All'):
    frequency = 'W'  
    days = int(periodPicker.value.split(' ')[0]) * 365
    wishedStart = dt.datetime.today() - pd.Timedelta(days, unit='d')
    if(pd.to_datetime(startPlot) < pd.to_datetime(wishedStart)):
      startPlot = wishedStart.strftime('%Y-%m-%d')   
    # df = df[df['DATE'] >= startPlot]    

monthList = pd.date_range(start=startPlot, end=dt.datetime.today(), freq=frequency).format(formatter=lambda x: x.strftime('%Y-%m-%d'))
monthList.append(dt.datetime.today().strftime('%Y-%m-%d'))
performanceList = []
for i, month in enumerate(monthList):
  p = PerformanceBlueprint(prcReader, df, month)
  p.calc()
  performanceList.append([p.date, p.equity, p.cost, p.realizedProfit, p.div, p.paperProfit, p.profit, p.profitRate, p.expense, p.ibov, p.sp500])

histProfDF = pd.DataFrame(performanceList, columns=['Date', 'Equity', 'Cost', 'Profit', 'Div', 'paperProfit', 'TotalProfit', '%Profit', 'Expense', '%IBOV', '%SP500'])
histProfDF['Date'] = pd.to_datetime(histProfDF.Date, format='%Y/%m/%d')
# if (startTimePicker.value == "12 months"):
histProfDF['%IBOV']   = histProfDF['%IBOV'] - histProfDF.iloc[0]['%IBOV']
histProfDF['%SP500']  = histProfDF['%SP500'] - histProfDF.iloc[0]['%SP500']
histProfDF['%Profit'] = histProfDF['%Profit'] - histProfDF.iloc[0]['%Profit']
# histProfDF

In [ ]:
import matplotlib.ticker as mtick

width = 2 if frequency == 'W' else 5     # the width of the bars: can also be len(x) sequence
shift = pd.Timedelta(width/2, unit='d')
fig, ax = plt.subplots(2, 1, figsize=(32, 9), sharex=True, gridspec_kw={'height_ratios': [3, 1]})
fig.tight_layout()

ax[0].plot(histProfDF.Date, histProfDF['%IBOV'], label='ibovespa')
ax[0].plot(histProfDF.Date, histProfDF['%SP500'], label='S&P500')
ax[0].plot(histProfDF.Date, histProfDF['%Profit'], label='Wallet')

minTick = min(histProfDF['%IBOV'].min(), histProfDF['%SP500'].min(), histProfDF['%Profit'].min())
maxTick = max(histProfDF['%IBOV'].max(), histProfDF['%SP500'].max(), histProfDF['%Profit'].max())

ax[0].set_yticks(np.arange(minTick, maxTick, 0.03))
ax[0].axhline(y=0, color='k')
ax[0].grid(True, which='both')
ax[0].yaxis.set_major_formatter(mtick.PercentFormatter(1))
ax[0].legend()

barsDf = histProfDF[:-1]
# ax[1].grid(True, which='both')
ax[1].bar(barsDf.Date - shift, barsDf['Equity'], width, label='Equity')
ax[1].bar(barsDf.Date - shift, barsDf['Div']   , width, bottom=barsDf['Equity'], label='Div')
ax[1].bar(barsDf.Date - shift, barsDf['Profit'], width, bottom=barsDf['Div'] + barsDf['Equity'], label='Profit')
ax[1].bar(barsDf.Date + shift, barsDf['Cost']  , width, label='Cost')
ax[1].legend()
ax[1].set_ylabel('R$')

plt.xticks(barsDf['Date'], rotation=90)
plt.title('Historical profitability')
plt.show()

# Dividends by month

In [ ]:
m = int(dt.datetime.today().strftime("%m"))
y = int(dt.datetime.today().strftime("%Y"))
# m=9

divTable = divReader.df
prov = df[df['OPERATION'] == 'D']

if(not (divTable.empty & prov.empty)):
    divTable = divTable.reset_index()
    divTable = divTable[(divTable['PAYDATE'].dt.month == m) & (divTable['PAYDATE'].dt.year == y)]

    divTable= pd.merge(divTable, prov, how='inner', on=['PAYDATE', 'DATE', 'SYMBOL', 'PRICE'])
        
    divTable = divTable[['PAYDATE', 'SYMBOL','AMOUNT']]
    divTable.columns = ['DATE', 'PAYDATE', 'R$']
    # divTable = divTable.groupby(['Ativo', 'DATE'])['R$'].sum().reset_index()
    # display(divTable)
    divTable.sort_values('DATE', inplace=True)
    divTable['DATE'] = divTable['DATE'].apply(lambda x: x.strftime('%Y-%m-%d'))
    divTable.loc['Total', 'R$'] = divTable['R$'].sum()
    divTable.fillna(' ', inplace=True)
    display(divTable)

In [ ]:
if(not (divTable.empty & prov.empty)):
    pvt = prov.pivot_table(index='Year', columns='Month', values='AMOUNT', margins=True, margins_name='Total', aggfunc='sum', fill_value=0)
    sorted_m = sorted(pvt.columns[:-1], key=lambda month: dt.datetime.strptime(month, "%B"))
    sorted_m.append(pvt.columns[-1])
    display(pvt.reindex(sorted_m, axis=1))

In [ ]:
df[df['SYMBOL'].isin(['SPTW11'])]